
<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px">
📌
Importing modules
    </div>

In [1]:
!pip install transformers

In [2]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from sklearn.model_selection import train_test_split


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px">
📌
Loading the data    </div>

In [3]:
prompts_train = pd.read_csv("/content/prompts_train.csv")
prompts_test = pd.read_csv("/content/prompts_test.csv")

summaries_train = pd.read_csv("/content/summaries_train.csv")
summaries_test = pd.read_csv("/content/summaries_test.csv")

sample_submission = pd.read_csv("/content/sample_submission.csv")

prompts_train

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


In [4]:
train = summaries_train.merge(prompts_train, how="left", on="prompt_id")#[:2000]
test = summaries_test.merge(prompts_test, how="left", on="prompt_id")

train

,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
...,...,...,...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...,0.205683,0.380538,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...,-0.308448,0.048171,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...,-1.408180,-0.493603,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...,-0.393310,0.627128,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...


In [5]:
train.shape

(7165, 8)


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px">
📌
Preprocessing the data    </div>

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer.batch_encode_plus(
    train['text'].tolist(),
    truncation=True,
    padding=True
)

test_encodings = tokenizer.batch_encode_plus(
    test['text'].tolist(),
    truncation=True,
    padding=True
)

train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train['content'].tolist()),
    torch.tensor(train['wording'].tolist())
)

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask'])
)


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px">
📌
Defining the BERT model    </div>

In [8]:
from transformers import AutoModel

from transformers import AutoModel
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modified BERTModel
class ModifiedBERTModel(nn.Module):
    def __init__(self):
        super(ModifiedBERTModel, self).__init__()
        self.bert = AutoModel.from_pretrained('bert-base-uncased')

        # Increased dropout rate
        self.dropout = nn.Dropout(0.3)

        # Directly connecting BERT output to a 2-neuron layer without an intermediate hidden layer
        self.linear = nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        output = self.linear(pooled_output)
        return output

# Instantiate the modified model
model = ModifiedBERTModel().to(device)

# Keep the same optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Use the ReduceLROnPlateau learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.1)

# Keep the same loss function
criterion = nn.MSELoss()

model, optimizer, scheduler


(ModifiedBERTModel(
   (bert): BertModel(
     (embeddings): BertEmbeddings(
       (word_embeddings): Embedding(30522, 768, padding_idx=0)
       (position_embeddings): Embedding(512, 768)
       (token_type_embeddings): Embedding(2, 768)
       (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): BertEncoder(
       (layer): ModuleList(
         (0-11): 12 x BertLayer(
           (attention): BertAttention(
             (self): BertSelfAttention(
               (query): Linear(in_features=768, out_features=768, bias=True)
               (key): Linear(in_features=768, out_features=768, bias=True)
               (value): Linear(in_features=768, out_features=768, bias=True)
               (dropout): Dropout(p=0.1, inplace=False)
             )
             (output): BertSelfOutput(
               (dense): Linear(in_features=768, out_features=768, bias=True)
               (LayerNorm): LayerNorm((768,), 


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px">
📌
Training the BERT model    </div>


<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px">
📌
Creating data loader and performing sanity check    </div>

In [9]:
batch_size= 16

In [10]:
# Splitting training data into train and validation sets
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2, random_state=0)
# Creating train loader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# Creating validation loader

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)




<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px">
📌
Training the model for 30 epochs    </div>

In [12]:
def train_with_early_stopping(train_loader, val_loader, model, optimizer, criterion, scheduler, device, n_epochs=10, patience=3):
    best_val_loss = float('inf')
    no_improvement_epochs = 0

    for epoch in range(n_epochs):
        running_loss = 0.0
        model.train()

        # Training loop for this epoch
        for step, (input_ids, attention_mask, content, wording) in enumerate(train_loader):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            content = content.to(device)
            wording = wording.to(device)

            optimizer.zero_grad()

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs[:, 0], content) + criterion(outputs[:, 1], wording)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validation loop for this epoch
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for val_step, (input_ids, attention_mask, content, wording) in enumerate(val_loader):
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                content = content.to(device)
                wording = wording.to(device)

                val_outputs = model(input_ids, attention_mask)
                val_loss += criterion(val_outputs[:, 0], content) + criterion(val_outputs[:, 1], wording)

        avg_val_loss = val_loss / len(val_loader)
        scheduler.step(avg_val_loss)  # Adjust learning rate

        # Check for early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            no_improvement_epochs = 0
        else:
            no_improvement_epochs += 1

        if no_improvement_epochs >= patience:
            print(f"Stopping early after {epoch+1} epochs!")
            break

    return model

# Note: In your environment, you can call the function with the necessary data loaders to train the model
model = train_with_early_stopping(train_loader, val_loader, model, optimizer, criterion, scheduler, device)



Stopping early after 5 epochs!



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px">
📌
Creating test loader    </div>

In [13]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px">
📌
Generating predictions on test set    </div>

In [14]:
model.eval()
predictions = []
with torch.no_grad():
    for input_ids, attention_mask in test_loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        outputs = model(input_ids, attention_mask)
        predictions.extend(outputs.cpu().numpy())



<div style="background-color:#F0E3D2; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px">
📌
Generating submission
    </div>

In [15]:
submission_df = pd.DataFrame({
    'student_id': test['student_id'],
    'content': [pred[0] for pred in predictions],
    'wording': [pred[1] for pred in predictions]
})

submission_df.to_csv('submission.csv', index=False)

In [16]:
submission_df

,student_id,content,wording
0,000000ffffff,-1.220712,-1.105185
1,111111eeeeee,-1.321865,-1.165090
2,222222cccccc,-1.287542,-1.184546
3,333333dddddd,-1.297873,-1.172369
